## Final Project Submission

Please fill out:
* Student name: Eduardo Calzadilla
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


# Movie genre and title recommendations

#### Datasets: 
    1. IMDB
    2. Rotten Tomatoes
    3. TheMovieDB.org
    4. Box Office Mojo





## Contents

**A. [Exploratory Data Analysis](#EDA)** <br>
  1. [Initial exploration and cleanup](#Iec)<br>
  2. [Exploratory visualizations](#Ev)<br>
  3. [Transformations and joins](#Tj)<br>
  
**B.**
 

## Introduction & Summary

Throughout this notebook we will be exploring three datasets as highlighted above, in order to spot trend that could be indicative of higher succes capacity and return on investment (ROI). 

We will be focusing on the following four null hypotheses:
  1. Movies with higher ratings do not perform significantly better in terms of gross income and ROI.
  2. All genres perform equally in terms of ratings, gross income and ROI.
  3. There are no words that are indicative of higher performance in terms of ratings and ROI.
  4. There has been no change in terms of competition (number of movies coming out) 4 year prior to 2015 and 4 year after 2015. 

We have found that there are significant similarities between median ratings and median performance. etc...

In [96]:
# Importing necessary packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import glob
import zipfile
%matplotlib inline

In [8]:
# Passing zip doc filenames to a list for storing DataFrames. 
zippedData = glob.glob("zippedData/*.gz")

In [206]:
# Verifyig that all the docs in zippedData were read correctly
zippedData

['zippedData/imdb.title.crew.csv.gz',
 'zippedData/tmdb.movies.csv.gz',
 'zippedData/imdb.title.akas.csv.gz',
 'zippedData/imdb.title.ratings.csv.gz',
 'zippedData/imdb.name.basics.csv.gz',
 'zippedData/rt.reviews.tsv.gz',
 'zippedData/imdb.title.basics.csv.gz',
 'zippedData/rt.movie_info.tsv.gz',
 'zippedData/tn.movie_budgets.csv.gz',
 'zippedData/bom.movie_gross.csv.gz',
 'zippedData/imdb.title.principals.csv.gz']

In [190]:
# Ceate list of variables for storing DataFrames.
df_name=[]
for filename in zippedData:
    df_name.append("df_" + filename.split("/",1)[1].split(".",3)[0] 
                + "_" + filename.split("/",1)[1].split(".",3)[1] 
                + "_" + filename.split("/",1)[1].split(".",3)[2])

In [199]:
df_dic={}
i = 0
for filename in zippedData:
    if "csv" in filename:
        df_dic[df_name[i]] = pd.read_csv(filename,
                             compression='gzip')
    else: 
        df_dic[df_name[i]] = pd.read_csv(filename
                                   , sep='\t'
                                   , encoding = 'unicode_escape')
    i+=1
    

In [201]:
df_imdb_title_crew = df_dic["df_imdb_title_crew"]
df_imdb_title_akas = df_dic["df_imdb_title_akas"]
df_imdb_title_ratings = df_dic["df_imdb_title_ratings"]
df_imdb_name_basics = df_dic["df_imdb_name_basics"]
df_imdb_title_basics = df_dic["df_imdb_title_basics"]
df_imdb_title_principals = df_dic["df_imdb_title_principals"]

df_rt_movie_info_tsv = df_dic["df_rt_movie_info_tsv"]
df_rt_reviews_tsv = df_dic["df_rt_reviews_tsv"]

df_tmdb_movies_csv = df_dic["df_tmdb_movies_csv"]

df_tn_movie_budgets_csv = df_dic["df_tn_movie_budgets_csv"]

df_bom_movie_gross_csv = df_dic["df_bom_movie_gross_csv"]


<a id="EDA"></a>

## EXPLORATORY DATA ANALYSIS

Now that the datastes are loaded into DataFrames, we will explore the datasets to understand the data present in each. Later we will drill into the data to understand possible null values and start cleaning the same for joining and use. 

As we see the data for the first time, we will make note of potential areas for cleanup.

In [127]:
df_imdb_title_crew.head(2)

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


As we see above the all columns seem to contain secondary keys, or it could also contain useless data. We will verify as we see other IMBD datasets, and considering possible joins.

In [130]:
df_imdb_title_akas.head(2)

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0


Above we see that this df contains movie titles, and lists (in binary) if the title is the original, as well as the region and language it was released in. We will find more about attribute values later on. For now we are assuming original titles are pre-release or location specific. 

We can also see that the "title id" column has a similar format to the "tconst" column in the previous df.

In [131]:
df_imdb_title_ratings.head(2)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


It now seems that the "tconst" column in the first df joins with this df rather than the "df_imdb_title_akas" df. And this df provides the ratings as well as number of votes for each score. 

In [207]:
df_imdb_name_basics.head(2)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


In this df we see data associated with crew members and their associated movies. Given the nature of our study, we will probably ignore this table altogether, unless it proves necessary for joining tables. 

In [208]:
df_imdb_title_basics.head(2)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


Here we see the genre data and that it can be joined to the "df_imdb_title_ratings" df. 

In [209]:
df_imdb_title_principals.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN


This df contains a joining column for all other columns, except the "df_imdb_title_akas" df. It provides parsed information on the roles of the cast and crew related dataframe. 

In [210]:
df_rt_reviews_tsv.head(2)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"


It seems the above dataframe contains review information, and names and publishing means of the critic (also read "movie-viewer") who posted the same. It could be joined to specific movie using the id column. 

We will verify if this is the case with the following df. 

In [212]:
df_rt_movie_info_tsv.head(2)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


Our original assumption seems correct in that the RottenTomatoes DataFrames can be joined using the "id" column in the previous table. However, we do not have the name of the film which might be useful for our word count analysis - however, it might not be necessary if we already have enough data samples from IMDB, for example.  

Additionally this dataframe contains "writer", "director", "dvd_date", and "studio" columns which will not be useful for our analysis, but the "director" and "writer" columns could be used for joining to movie titles. We will also need to further investigate the "rating" columns as it seems that the column in this table is specifying the Motion Picture Association (MPA) rating system, rather than point ratings associated with a review.

In [214]:
df_tmdb_movies_csv.head(2)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610


In the above we see that the "Unnamed:0" column can probably be deleted as it seems to contain an index rather than the primary key, as there is another column named "id". 

We can also probably delete one of the title columns. We will be keeping the "title" column, rather than the "original title" column, as we are making the assumption that the latter contains titles used while working on the movie (or prerelease titles).

In [148]:
df_tn_movie_budgets_csv.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [149]:
df_bom_movie_gross_csv.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In the above we can see both dataframes contain financial data.

We don't need both dataframes and will choose the former as it contains the full release date and the production budget. Both will be useful for our analysis, and the exact release date can allow us more granularity on when to release Microsoft's first film.  

<a id="Iec"></a>
## Initial exploration and cleanup

Let's take a quick recap on we've learned from each dataset so far. 


In [ ]:
Here we see potentially incorrect datatypes as well as some missing values. I'll first check number of missing values to see if it is wise to drop these rows.

In [ ]:
# Your code here - remember to use markdown cells for comments as well!